### ICD-11 Downloads

#### info

By default one client Id and client secret is provided to be able to authenticate your software to use the ICD APIs. You may however, add more clients or remove some of them.
(IMPORTANT!!! If a client key is removed, clients configured with this Id/secret will not be able access the API anymore)

----------------

```
config.json
-----------------
{
ClientId: "XXXX",
ClientSecret: "XXXX"
}
```

------------------

In [1]:
import requests
import json

### Token Set up

In [2]:
with open("config.json", 'r')as f:
    config = json.load(f)

In [3]:
'''Set data to post'''
payload = {'client_id': config["ClientId"],\
           'client_secret': config["ClientSecret"],\
           'scope': 'icdapi_access',\
           'grant_type': 'client_credentials'}

           
'''make request'''
token_endpoint = 'https://icdaccessmanagement.who.int/connect/token'
r = requests.post(token_endpoint,\
                  data=payload,\
                  verify=True).json()

'''get token'''
token = r['access_token']

### Access ICD Download API

In [4]:
'''HTTP header fields to set'''
headers = {'Authorization':  'Bearer '+token, 
           'Accept': 'application/json', 
           'API-Version':'v2',
           'Accept-Language': 'en'}

In [5]:
def download(headers,url):
    '''make request '''          
    r = requests.get(url, headers=headers, verify=True)
    return r.json()

##### Sample Download

In [6]:
'''Define id term'''
foundation_id = 'https://id.who.int/icd/entity/'

'''Implement the foundation search'''
result = download(headers,foundation_id)

'''Dump the result'''
with open('samples/result.json', 'w')as f:
    json.dump(result,f)

In [9]:
result

{'@context': 'http://id.who.int/icd/contexts/contextForTopLevel.json',
 '@id': 'http://id.who.int/icd/entity',
 'browserUrl': 'NA',
 'child': ['http://id.who.int/icd/entity/1435254666',
  'http://id.who.int/icd/entity/1630407678',
  'http://id.who.int/icd/entity/1766440644',
  'http://id.who.int/icd/entity/1954798891',
  'http://id.who.int/icd/entity/21500692',
  'http://id.who.int/icd/entity/334423054',
  'http://id.who.int/icd/entity/274880002',
  'http://id.who.int/icd/entity/1296093776',
  'http://id.who.int/icd/entity/868865918',
  'http://id.who.int/icd/entity/1218729044',
  'http://id.who.int/icd/entity/426429380',
  'http://id.who.int/icd/entity/197934298',
  'http://id.who.int/icd/entity/1256772020',
  'http://id.who.int/icd/entity/1639304259',
  'http://id.who.int/icd/entity/1473673350',
  'http://id.who.int/icd/entity/30659757',
  'http://id.who.int/icd/entity/577470983',
  'http://id.who.int/icd/entity/714000734',
  'http://id.who.int/icd/entity/1306203631',
  'http://id.wh

--------------

### Recursive Download

In [7]:
def recursive_download(headers,jfile,tfile,logfile,walk_path,foot_print,url):
    
    '''Find ID'''
    walk_id = url.split("/")[-1]
    result = download(headers,url)
    
    logfile.write("Downloading : " + url)
    logfile.write("\n")
    
    walk_path = walk_path+"."+ walk_id
    foot_print = foot_print + " - "
    
    print(foot_print  + url.split("/")[-1])
    
    
    '''Dump data to json file'''
    json.dump(result,jfile)
    jfile.write("\n")
    
    
    '''Write down all ID and titles'''
    tfile.write(walk_id + "|" + walk_path)
    tfile.write("\n")
    
    #print(walk_path,walk_id)
    
    
    '''Recursion over all tree branches'''
    if 'child' in result.keys():
        child_urlset = result['child']
        if len(child_urlset)>0:
            for child_url in child_urlset:
                recursive_download(headers,jfile,tfile,logfile,walk_path,foot_print,child_url)
    return 

### Implementation of Recursive Download

In [8]:
root_url = 'https://id.who.int/icd/entity/21500692'
root_id = root_url.split("/")[-1]
tfile = open("data/" + root_id + ".txt",'w')
jfile = open("data/" + root_id + ".json",'w')
logfile = open("data/log_" + root_id + ".json",'w')
walk_path = root_id
foot_print = root_id

recursive_download(headers,jfile,tfile,logfile,walk_path,foot_print,root_url)

21500692 - 21500692
21500692 -  - 461716838
21500692 -  -  - 893358872
21500692 -  -  -  - 1978381958
21500692 -  -  -  -  - 1076031454
21500692 -  -  -  -  - 1148879218
21500692 -  -  -  -  - 1705900796
21500692 -  -  -  -  - 12250423
21500692 -  -  -  -  - 1248064204
21500692 -  -  -  - 1290583069
21500692 -  -  -  -  - 698839433
21500692 -  -  -  -  -  - 807359335
21500692 -  -  -  -  -  - 174958079
21500692 -  -  -  -  - 2121003176
21500692 -  -  -  -  -  - 183314925
21500692 -  -  -  -  -  - 1739480922
21500692 -  -  -  -  - 455097662
21500692 -  -  -  -  -  - 87152125
21500692 -  -  -  -  - 1871539651
21500692 -  -  -  -  -  - 172628415
21500692 -  -  -  -  -  - 1489672789


KeyboardInterrupt: 

In [ ]:
def recursive(i,tfile,jfile,title,xfile,item):
    
    rr = requests.get(item, headers=headers, verify=True)
    
    try:
        ll = json.loads(str(rr.text.strip()))
    
    
        if i%100 == 0:
            print(i,"done...")
        
        '''Dump data to json file'''
        json.dump(ll,jfile)
        jfile.write("\n")
    
    
        '''Find ID and Title'''
        ID = item.split("/")[-1]
        title = title + " | " + ll['title']["@value"]
    
    
        '''Write down all ID and titles'''
        tfile.write(ID + title)
        tfile.write("\n")
        #print(ID,title)
    
    
        '''Recursion over all tree branches'''
        if 'child' in ll.keys():
            new_idlist = ll['child']
            if len(new_idlist)>0:
                for term in new_idlist:
                    i = i+1
                    recursive(i,tfile,jfile,title,xfile,term)
                    
    except json.decoder.JSONDecodeError:
        print(item)
        '''Write down all ID and titles'''
        xfile.write(item)
        xfile.write("\n")
        
    return 

In [5]:
uri = 'https://id.who.int/icd/entity'

# HTTP header fields to set
headers = {'Authorization':  'Bearer '+token, 
           'Accept': 'application/json', 
           'Accept-Language': 'en'}
           
# make request           
r = requests.get(uri, headers=headers, verify=True)

# print the result
# print (r.text) 

In [6]:
print(r.text)

{"@context":{ 
                "title": "http://www.w3.org/2004/02/skos/core#prefLabel",
                "definition": "http://www.w3.org/2004/02/skos/core#definition",
                "child": "http://www.w3.org/2004/02/skos/core#narrowerTransitive",
                "synonym": "http://www.w3.org/2004/02/skos/core#altLabel",
                "browserUrl": "http://id.who.int/icd/schema/browserUrl",
                "releaseDate":"http://id.who.int/icd/schema/releaseDate "
             },"@id":"http://id.who.int/icd/entity","title":{"@language":"en","@value":"International Classification of Diseases 11th Revision"},"releaseDate":"2018-06-05","child":["http://id.who.int/icd/entity/1435254666","http://id.who.int/icd/entity/1630407678","http://id.who.int/icd/entity/1766440644","http://id.who.int/icd/entity/1954798891","http://id.who.int/icd/entity/21500692","http://id.who.int/icd/entity/334423054","http://id.who.int/icd/entity/274880002","http://id.who.int/icd/entity/1296093776","http://id.wh

In [7]:
def recursive(i,jfile,item):
    
    rr = requests.get(item, headers=headers, verify=True)
    ll = json.loads(str(rr.text.strip()))
    
    
    if i%100 == 0:
        print(i,"done...")
        
    '''Dump data to json file'''
    json.dump(ll,jfile)
    jfile.write("\n")
    
    
    '''Recursion over all tree branches'''
    if 'child' in ll.keys():
        new_idlist = ll['child']
        if len(new_idlist)>0:
            for term in new_idlist:
                i = i+1
                recursive(i,jfile,term)
        
    return i

In [9]:
data = json.loads(r.text.strip())
idlist = data['child']
title = ''
i = 0

url = 'http://id.who.int/icd/entity/979408586'

#url = item
ID = url.split("/")[-1]
jfile = open("data/" + ID + ".json",'w')
    
print(url,"started....")
recursive(i,jfile,url)
print(url,"completed....")  

http://id.who.int/icd/entity/979408586 started....
0 done...
100 done...
200 done...
100 done...


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

01== Y ["http://id.who.int/icd/entity/1435254666",
02== Y "http://id.who.int/icd/entity/1630407678",
03== Y "http://id.who.int/icd/entity/1766440644",
04== Y "http://id.who.int/icd/entity/1954798891",
05== Y "http://id.who.int/icd/entity/21500692",
06== Y "http://id.who.int/icd/entity/334423054",
07== Y "http://id.who.int/icd/entity/274880002",



08"N  http://id.who.int/icd/entity/1296093776",
09"Y  http://id.who.int/icd/entity/868865918",
10"Y  http://id.who.int/icd/entity/1218729044",
11"Y  http://id.who.int/icd/entity/426429380",
12"Y  http://id.who.int/icd/entity/197934298",
13"Y  http://id.who.int/icd/entity/1256772020",
  

14"N  http://id.who.int/icd/entity/1639304259",
15"N  http://id.who.int/icd/entity/1473673350",
16"Y  http://id.who.int/icd/entity/30659757",
17"Y  http://id.who.int/icd/entity/577470983",
18"Y  http://id.who.int/icd/entity/714000734",
19"Y  http://id.who.int/icd/entity/1306203631",
20"N  http://id.who.int/icd/entity/223744320",


21"Y  http://id.who.int/icd/entity/1843895818",
22"Y  http://id.who.int/icd/entity/435227771",
23"Y  http://id.who.int/icd/entity/850137482",
24"Y  http://id.who.int/icd/entity/1249056269",
25"Y  http://id.who.int/icd/entity/1596590595",
26"Y  http://id.who.int/icd/entity/718687701",
27"Y  http://id.who.int/icd/entity/231358748",
28"N  http://id.who.int/icd/entity/979408586",
29"N  http://id.who.int/icd/entity/1801349023"]

In [ ]:
import json
line = json.loads(r.text.strip())
for uri in line['child']:
    #print(item)
    rr = requests.get(uri, headers=headers, verify=True)
    ll = json.loads(rr.text.strip())
    if 
    for u in ll['child']:
        print(u)    

In [10]:
uri = 'https://id.who.int/icd/entity/1766440644'

In [ ]:
for item in r['child']:
    print(item)

In [16]:
# make request           
r = requests.get(uri, headers=headers, verify=True)

# print the result
print (r.text) 

{"@context":{ 
                "title": "http://www.w3.org/2004/02/skos/core#prefLabel",
                "definition": "http://www.w3.org/2004/02/skos/core#definition",
                "child": "http://www.w3.org/2004/02/skos/core#narrowerTransitive",
                "synonym": "http://www.w3.org/2004/02/skos/core#altLabel",
                "browserUrl": "http://id.who.int/icd/schema/browserUrl",
                "releaseDate":"http://id.who.int/icd/schema/releaseDate "
             },"@id":"http://id.who.int/icd/entity","title":{"@language":"en","@value":"International Classification of Diseases 11th Revision"},"releaseDate":"2018-06-05","child":["http://id.who.int/icd/entity/1435254666","http://id.who.int/icd/entity/1630407678","http://id.who.int/icd/entity/1766440644","http://id.who.int/icd/entity/1954798891","http://id.who.int/icd/entity/21500692","http://id.who.int/icd/entity/334423054","http://id.who.int/icd/entity/274880002","http://id.who.int/icd/entity/1296093776","http://id.wh

In [ ]:
temporal relationship: 
distance interpretation in graph
symptoms based text-mining => therapy

In [ ]:
Physician-AI system 

In [ ]:
icd codes = 55,000
14,000 changes

In [ ]:
Mesh => Dignostic